In [117]:
library(rio)
library(dplyr)
library(tidyr)
library(srvyr)
library(stringr)

options(survey.lonely.psu = "adjust")

## Import Master yang Dibutuhkan

1. Master Area : Luas LBS per Kecamatan - Strata
2. Master Segmen : Master Sampel Segmen di setiap bulannya

In [ ]:
master_area = import('/data/ksa/00_Data_Input/estimation_data/master_area.Rds')

In [ ]:
master_segmen = import('/data/ksa/00_Data_Input/estimation_data/master_segmen.Rds')

## Contoh Format Data Input

Keterangan:

1. id_segmen : ID segmen 
2. subsegmen : Kode subsegmen (A1, A2, A3, B1, B2, B3, C1, C2, C3)
3. tahun : 4 digit kode tahun (2022, 2023)
4. bulan : 2 digit bulan (
5. fase_obs : fase berdasarkan amatan ksa
6. fase_pred : fase prediksi titik
7. fase_dom : fase prediksi berdasarkan yang dominan
8. periode : kode periode, 6 digit berisi 4 digit tahun dan 2 digit bulan

In [118]:
input_data_amatan = import('/data/ksa/00_Data_Input/estimation_data/input_data_amatan.Rds')
input_data_amatan %>% head()

Warning message:
“Missing `trust` will be set to FALSE by default for RDS in 2.0.0.”


,id_segmen,subsegmen,tahun,bulan,fase_obs,fase_pred,fase_dom,periode
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>
1,320101006,A1,2022,01,2,1,2,202201
2,320101006,A2,2022,01,7,2,7,202201
3,320101006,A3,2022,01,7,7,7,202201
4,320101006,B1,2022,01,1,1,1,202201
5,320101006,B2,2022,01,2,0,7,202201
6,320101006,B3,2022,01,7,7,7,202201


## Daftar Functions

generate_survey_data : untuk persiapan data amatan

In [119]:
generate_survey_data = function(result, result_prev, master_segmen, fase_){
  res = result %>% 
    left_join(result_prev %>% select(id_segmen, subsegmen, fase_prev = !!sym(fase_)), by = c('id_segmen', 'subsegmen')) %>% 
    left_join(master_segmen, by = c('id_segmen', 'periode')) %>% 
    transmute(
      periode, 
      id_prov = substr(id_segmen,1,2),
      id_kab = substr(id_segmen,1,4),
      id_kec = substr(id_segmen,1,7),
      id_segmen,
      subsegmen,
      strati_fin = factor(strati_fin),
      fase = as.character(!!sym(fase_)),
      fase_prev = as.character(fase_prev)
    ) 
  
  return(res)
}

In [120]:
temp = generate_survey_data(
    result = input_data_amatan %>% filter(periode == "202204"),
    result_prev = input_data_amatan %>% filter(periode == "202203"),
    master_segmen = master_segmen,
    fase_ = "fase_obs"
)

In [121]:
div9 = function(x,div){
  return(x/div)
}

data_reshape = function(survey_data){
  data_amatan = survey_data %>% 
    mutate(
      fase_imp = case_when(
        fase=="1" & (fase_prev == "2" | fase_prev == "3") ~ "91",
        fase=="2" & (fase_prev == "2" | fase_prev == "3") ~ "92",
        fase=="5" & (fase_prev == "2" | fase_prev == "3") ~ "95",
        fase=="7" & (fase_prev == "2" | fase_prev == "3") ~ "97",
        fase=="4" & fase_prev=="4" ~ "0",
        fase=="6" & fase_prev=="6" ~ "0",
        .default  = as.character(fase)
      ),
      fase_imp = factor(fase_imp,c(0:7,91,92,95,97))
    ) 
  
  ctab = table(data_amatan$id_segmen,data_amatan$fase_imp)
  
  ctab = as.data.frame.matrix(ctab)
  
  if(nrow(ctab)>0){
    ctab_exp = ctab
    ctab_exp$id_segmen = rownames(ctab_exp)
    ctab_exp = ctab_exp %>% 
      transmute(
        V1 = `1`+`91`,
        V2 = `2`+`92`,
        G = `3`,
        panen = `4`,
        panen_antarsurvei = `91`+`92`+`95`+`97`,
        panen_total = `4`+`91`+`92`+`95`+`97`,
        bera = `0`,
        persiapan_lahan = `5`+`95`,
        non_vegetasi_dan_non_sawah = `7`+`97`,
        id_segmen = id_segmen
      ) 
    ctab_exp$ke = unique(survey_data$periode)  
  }
  
  ctab$rowsum = rowSums(ctab)
  
  ctab$id_segmen = rownames(ctab)
  
  ctab2 = ctab %>% mutate_if(is.numeric,div9,div=.$rowsum)
  ctab2$rowsum = NULL
  
  return(ctab2)
}

In [122]:
calc_weight = function(prov_id, survey_data, master_segmen, master_area, level_est){
  
  if(!(level_est %in% c(1,2,3))){
    stop("level_est hanya dapat bernilai 1 (provinsi), 2 (kabupaten), atau 3 (kecamatan)")
  }
    
  master_strati_segmen = unique(master_segmen$strati_fin)
  strati_survey = unique(survey_data$strati_fin)
  
  missing_strati_fin = data.frame(strati_fin = master_strati_segmen) %>% 
    mutate(
      class = substr(strati_fin,8,9),
      flag = ifelse(strati_fin %in% strati_survey,1,0)
    )
  
  master_area_adjusted = master_area %>% 
    select(id:nama,ls3,ls1s2gab) %>% 
    rename(
      "S1" = ls1s2gab,
      "S3" = ls3
    ) %>% 
    pivot_longer(
      !c(id:nama),
      names_to = "tipe",
      values_to = "luas"
    )  %>% 
    mutate(
      strati_fin = paste0(id_kec,tipe)
    ) %>% 
    right_join(missing_strati_fin,"strati_fin")
  
  master_area_used = master_area %>% filter(id_prov==prov_id)
  data_survei_used = survey_data %>% filter(id_prov==prov_id)
  
  if(level_est==1){
    master_area_for_adjust_used = master_area_adjusted %>% 
      group_by(id_prov,class) %>% 
      mutate(
        factor = sum(luas)/sum(luas[flag==1]),
        factor = ifelse(is.infinite(factor),0,factor)
      ) %>% 
      ungroup() %>% 
      select(strati_fin,factor)
    
  } else if(level_est==2){
    master_area_for_adjust_used = master_area_adjusted %>% 
      group_by(id_prov,id_kab,class) %>% 
      mutate(
        factor = sum(luas)/sum(luas[flag==1]),
        factor = ifelse(is.infinite(factor),0,factor)
      ) %>% 
      ungroup() %>% 
      select(strati_fin,factor)
    
  } else if(level_est==3){
    master_area_for_adjust_used = master_area_adjusted %>% 
      group_by(id_prov,id_kab,id_kec,class) %>% 
      mutate(
        factor = sum(luas)/sum(luas[flag==1]),
        factor = ifelse(is.infinite(factor),0,factor)
      ) %>% 
      ungroup() %>% 
      select(strati_fin,factor)
  }
  
  master_area_used = master_area_used %>% 
    filter(id_kec %in% unique(data_survei_used$id_kec)) %>% 
    select(id, id_prov, id_kab, id_kec,ls3,ls1s2gab) %>% 
    rename(
      "S1" = ls1s2gab,
      "S3" = ls3
    ) %>% 
    gather(key = "tipe",value = "luas",-id:-id_kec) %>% 
    mutate(
      strati_fin = paste0(id_kec,tipe),
      prob = ifelse(luas==0,0,1/luas)
    ) %>% 
    filter(strati_fin %in% unique(data_survei_used$strati_fin)) 
  
  data_segmen_used = data_survei_used %>% 
    select(periode, id_prov, id_kab, id_kec, id_segmen, strati_fin) %>% distinct() %>% 
    left_join(master_area_used %>% select(luas,strati_fin,prob),"strati_fin") %>% 
    left_join(master_area_for_adjust_used,"strati_fin") %>% 
    group_by(strati_fin) %>% 
    mutate(
      factor = ifelse(is.na(factor), 1, factor), #add
      n = n(),
      weight = ifelse(prob==0,0,1/(n*prob)*factor)
    ) %>% 
    select(-factor)
  
  ctab2 = data_reshape(survey_data = survey_data)
  
  data_amatan_segmen = data_segmen_used %>%
    left_join(ctab2,"id_segmen") %>% 
    mutate(
      V1 = `1`+`91`,
      V2 = `2`+`92`,
      G = `3`,
      panen = `4`,
      panen_antarsurvei = `91`+`92`+`95`+`97`,
      panen_total = `4`+`91`+`92`+`95`+`97`,
      bera = `0`,
      persiapan_lahan = `5`+`95`,
      non_vegetasi_dan_non_sawah = `7`+`97`,
    ) %>% 
    filter(!is.na(luas)) %>%  
    mutate(
      strati = substr(strati_fin, 8,9)
    )
  
  return(data_amatan_segmen)
}

In [123]:
hitung_rse = function(variable, grouping, est_type = "mean", desain, weight){
  
  if(!(est_type %in% c("total", "mean"))){
    stop("est_type hanya bernilai 'total' atau 'mean'")
  }
  
  if(est_type == "total"){
    temp = desain %>% 
      group_by_at(grouping) %>% 
      summarise(
        variable = survey_total(!!sym(variable), vartype = c("se", "var", "cv", "ci"), deff = T),
        variable_unweighted = sum(!!sym(variable)),
        variable_n = sum(!!sym(weight) > 0, na.rm = T),
        variable_N = sum(!!sym(weight), na.rm = T)
      ) 
  }else{
    temp = desain %>% 
      group_by_at(grouping) %>% 
      summarise(
        variable = survey_mean(!!sym(variable), vartype = c("se", "var", "cv", "ci"), deff = T),
        variable_unweighted = mean(!!sym(variable)),
        variable_n = sum(!!sym(weight) > 0, na.rm = T),
        variable_N = sum(!!sym(weight))
      ) 
  }
  
  colnames(temp) = gsub("variable", variable, colnames(temp))
  colnames(temp)[colnames(temp) == variable] = paste0(variable, "_est")
  
  output = temp %>% pivot_longer(!grouping) %>% 
    mutate(
      est_type = est_type,
      variable = sub("_(?!.*_).*", "", name, perl = TRUE),
      ind = gsub(".*_", "", name),
      ind = factor(ind, levels = c("unweighted", "est", "se", "var", "cv", "low", "upp", "deff", "n", "N"))
    ) %>% 
    select_at(c(grouping, "variable", "est_type", "ind","value")) %>% 
    arrange_at(c(grouping, "variable", "est_type", "ind")) %>% 
    pivot_wider(names_from = "ind", values_from = value) %>% 
    mutate(
      cv = cv*100
    )
  
  return(output)
}

estimasi = function(variable, prov_id, survey_data, master_segmen, master_area, level_est, est_type = "mean", est_strata = F, details = T){
  
  est_output = data.frame()
  for (lev_est in level_est) {
    
    if(lev_est == 1){
      grouping = "id_prov"
    }else{
      if(lev_est == 2){
        grouping = c("id_prov","id_kab")
      }else{
        if(lev_est == 3){
          grouping = c("id_prov", "id_kab", "id_kec")
        }else{
          stop("level_est hanya bernilai 1,2,3")
        }
      }
    }
    
    if(est_strata){grouping = c(grouping, "strati")}
    
    print("------")
    for (prov in prov_id) {
      survey_data_weighted = calc_weight(prov_id = prov_id, master_area = master_area, master_segmen = master_segmen, survey_data = survey_data, level_est = lev_est)
      
      des = survey_data_weighted %>% 
        mutate(
          strati_fin = factor(strati_fin)
        ) %>% 
        as_survey_design(
          ids = id_segmen,
          strata = strati_fin,
          weights = weight
        )
      
      
      for (est_type_ in est_type) {
        for (vari in variable) {
          if(details){
            print(paste0("Level Estimasi : ", lev_est, " | ID Provinsi : ", prov, " | Tipe Estimasi : ", str_pad(string = est_type_, width = 5, side = "left", pad = " "), " | Variable : ", vari))
          }
          temp = hitung_rse(variable = vari, grouping = grouping, est_type = est_type_, desain = des, weight = "weight")
          est_output = bind_rows(est_output, temp)
        }
      }
    }
  }
  
  
  est_output = est_output %>% select(starts_with("id_") | starts_with("strati")) %>% 
    cbind(
      est_output %>% select(!(starts_with("id_") | starts_with("strati")))
    )
  
  return(est_output)
}


In [124]:
list_periode_lengkap = sort(unique(input_data_amatan$periode))
list_periode = list_periode_lengkap[-1]
list_periode_lengkap

[1] "202201" "202202" "202203" "202204" "202205" "202206" "202207" "202208"
 [9] "202209" "202210" "202211" "202212" "202301" "202302" "202303" "202304"
[17] "202305" "202306" "202307" "202308" "202309" "202310" "202311" "202312"

In [125]:
fase_list = c("fase_obs", "fase_pred", "fase_dom")

In [128]:
for (fase_temp in fase_list[1]) {
    
  all_est = data.frame()
  for (periode_ in list_periode[1:2]) {
    print(paste0(fase_temp, " ", periode_))

    periode_ke = c(1:length(list_periode_lengkap))[list_periode_lengkap == periode_]  
    periode_prev_ = periode_ke - 1
    periode_prev = list_periode_lengkap[periode_prev_]
      
    data_survei = generate_survey_data(
      result = input_data_amatan %>% filter(periode == periode_), 
      result_prev = input_data_amatan %>% filter(periode == periode_prev),
      master_segmen = master_segmen %>% filter(periode == periode_), 
      fase_ = fase_temp
    )
    
    est1 = estimasi(
      variable = c( "bera", "V1", "V2", "G", "panen", "persiapan_lahan", "non_vegetasi_dan_non_sawah"),
      prov_id = 32,
      survey_data = data_survei,
      master_segmen = master_segmen %>% filter(periode == periode_),
      master_area = master_area,
      level_est = c(1, 2),
      est_type = c("mean", "total"),
      est_strata = F,
      details = T
    ) %>% 
      mutate(
        periode = periode_
      )
    
    est2 = estimasi(
      variable = c( "bera", "V1", "V2", "G", "panen", "panen_antarsurvei", "panen_total", "persiapan_lahan", "non_vegetasi_dan_non_sawah"),
      prov_id = 32,
      survey_data = data_survei,
      master_segmen = master_segmen %>% filter(periode == periode_),
      master_area = master_area,
      level_est = c(1, 2),
      est_type = c("mean", "total"),
      est_strata = T,
      details = T
    ) %>% 
      mutate(
        periode = periode_
      )
    
    est_ = bind_rows(est2, est1) %>% 
      replace_na(
        list(id_kab = "3200", strati = "ALL")
      ) %>% 
      mutate(variable = factor(variable, levels = c("bera", "V1", "V2", "G", "panen", "panen_antarsurvei", "panen_total", "persiapan_lahan", "non_vegetasi_dan_non_sawah"))) %>% 
      arrange(id_prov, id_kab, strati, variable)
    
    all_est = rbind(all_est, est_)
  }
  
  export(all_est, paste0(gsub(":", "_", Sys.time()), " ", fase_temp, ".xlsx"))
}

[1] "fase_obs 202202"
[1] "------"
[1] "Level Estimasi : 1 | ID Provinsi : 32 | Tipe Estimasi :  mean | Variable : bera"
[1] "Level Estimasi : 1 | ID Provinsi : 32 | Tipe Estimasi :  mean | Variable : V1"
[1] "Level Estimasi : 1 | ID Provinsi : 32 | Tipe Estimasi :  mean | Variable : V2"
[1] "Level Estimasi : 1 | ID Provinsi : 32 | Tipe Estimasi :  mean | Variable : G"
[1] "Level Estimasi : 1 | ID Provinsi : 32 | Tipe Estimasi :  mean | Variable : panen"
[1] "Level Estimasi : 1 | ID Provinsi : 32 | Tipe Estimasi :  mean | Variable : persiapan_lahan"
[1] "Level Estimasi : 1 | ID Provinsi : 32 | Tipe Estimasi :  mean | Variable : non_vegetasi_dan_non_sawah"
[1] "Level Estimasi : 1 | ID Provinsi : 32 | Tipe Estimasi : total | Variable : bera"
[1] "Level Estimasi : 1 | ID Provinsi : 32 | Tipe Estimasi : total | Variable : V1"
[1] "Level Estimasi : 1 | ID Provinsi : 32 | Tipe Estimasi : total | Variable : V2"
[1] "Level Estimasi : 1 | ID Provinsi : 32 | Tipe Estimasi : total | Variable : G"